# 0.0. IMPORTS

In [45]:
import sys
import psycopg2
import pandas  as pd
import numpy   as np
import seaborn as sns

## 0.1. Aux Functions

## 0.2. Data collection

In [3]:
# connection = psycopg2.connect( user = 'member',
#                                password = 'cdspa',
#                                host = 'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com',
#                                port = '5432',
#                                database = 'comunidadedsdb' )

# cursor = connection.cursor()

# query = """ SELECT *
#             FROM pa004.users
#             LEFT JOIN pa004.insurance on ( users.id = insurance.id )
#             LEFT JOIN pa004.vehicle on ( users.id = vehicle.id ) """

# cursor.execute( query )

# table = cursor.fetchall()

# outputquery = 'copy ({0}) to stdout with csv header'.format( query )

# with open( 'dataset.csv', 'w' ) as f:
#     cursor.copy_expert( outputquery, f )

# cursor.close()
# connection.close()

In [4]:
df_raw = pd.read_csv( 'train_dataset.csv' )
df_raw = df_raw.drop( df_raw.columns[[ 5, 10 ]], axis=1 )

# 1.0. DATA DESCRIPTION

In [5]:
df1 = df_raw.copy()

## Data Dimensions

In [9]:
print( 'Number of rows: {}'.format( df1.shape[0] ) )
print( 'Number of columns: {}'.format( df1.shape[1] ) )

Number of rows: 381109
Number of columns: 12


## Data types

In [10]:
df1.dtypes

id                       int64
gender                  object
age                      int64
region_code              int64
policy_sales_channel     int64
previously_insured       int64
annual_premium           int64
vintage                  int64
response                 int64
driving_license          int64
vehicle_age             object
vehicle_damage          object
dtype: object

## NA check

In [11]:
df1.isna().sum()

id                      0
gender                  0
age                     0
region_code             0
policy_sales_channel    0
previously_insured      0
annual_premium          0
vintage                 0
response                0
driving_license         0
vehicle_age             0
vehicle_damage          0
dtype: int64

## Descriptive Statistics

In [51]:
df_num = df1.select_dtypes( include=[ 'int64'] )

### Numerical attributes

In [52]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# central tendency - mean, median
ct1 = pd.DataFrame( df_num.apply( lambda x: np.mean( x ) ) ).T
ct2 = pd.DataFrame( df_num.apply( lambda x: np.median( x ) ) ).T

# dispersion metrics - min, max, range, std, skewness, kurtosis
dm1 = pd.DataFrame( df_num.apply( lambda x: np.min( x ) ) ).T
dm2 = pd.DataFrame( df_num.apply( lambda x: np.max( x ) ) ).T
dm3 = pd.DataFrame( df_num.apply( lambda x: np.max( x ) - np.min( x ) ) ).T
dm4 = pd.DataFrame( df_num.apply( lambda x: np.std( x ) ) ).T
dm5 = pd.DataFrame( df_num.apply( lambda x: x.skew() ) ).T
dm6 = pd.DataFrame( df_num.apply( lambda x: x.kurtosis() ) ).T

# concat
df_description = pd.concat( [ dm1, dm2, dm3, ct1, ct2, dm4, dm5, dm6 ] ).T
df_description.columns = ( 'min', 'max', 'range', 'mean', 'median', 'std', 'skewness', 'kurtosis' )
df_description

,min,max,range,mean,median,std,skewness,kurtosis
id,1.000,381109.000,381108.000,190555.000,190555.000,110016.692,-0.000,-1.200
age,20.000,85.000,65.000,38.823,36.000,15.512,0.673,-0.566
region_code,0.000,52.000,52.000,26.389,28.000,13.230,-0.115,-0.868
policy_sales_channel,1.000,163.000,162.000,112.034,133.000,54.204,-0.900,-0.971
previously_insured,0.000,1.000,1.000,0.458,0.000,0.498,0.168,-1.972
annual_premium,2630.000,540165.000,537535.000,30564.390,31669.000,17213.132,1.766,34.005
vintage,10.000,299.000,289.000,154.347,154.000,83.671,0.003,-1.201
response,0.000,1.000,1.000,0.123,0.000,0.328,2.302,3.299
driving_license,0.000,1.000,1.000,0.998,1.000,0.046,-21.595,464.354


# 2.0. FEATURE ENGINEERING